In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath, splitext
from pathlib import Path
from functools import partial
from itertools import product
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which sis why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fix_path(get_cwd('ignore_inspect_corr.ipynb', 'recon' +sep))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask import delayed, compute
from numba import jit, vectorize, float64, uint
from sortedcontainers import SortedList, SortedSet 

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option('display.max_rows', 500)
# pd.set_option('display.height', 500)
pd.set_option('display.max_columns', 50)

from common_util import DT_HOURLY_FREQ, DT_CAL_DAILY_FREQ, DT_BIZ_DAILY_FREQ, list_get_dict, inner_join, right_join, get_custom_biz_freq_df, query_df, search_df, chained_filter, recursive_dict, benchmark
from common_util import RECON_DIR, load_df, load_json, abs_df, df_count, df_value_count, outer_join, left_join, getcon, string_df_join_to_ser, is_df, is_ser, count_nn_nz_df, pairwise, cust_count, get_pardir_from_path
from data.data_api import DataAPI
from data.access_util import df_getters as dg, col_subsetters2 as cs2
from recon.common import REPORT_DIR, DATASET_DIR, default_gta_dataset, default_gta_test

## Library Functions

### Index Value Lookup

In [22]:
def value_lookup(lookup_obj):
    return {
        is_df(lookup_obj): lambda ser: ser.map(lambda idx: lookup_obj.loc[idx, ser.name]),
        is_ser(lookup_obj): lambda ser: ser.map(lambda idx: lookup_obj.loc[idx]),
        None: None
    }.get(True, None)

def index_lookup_dfs(idx_df, *lookup_tables):
    """
    Return index dataframe mapped to values in series or dataframe lookup tables.
    """
    yield from (idx_df.apply(value_lookup(lookup_table), axis=0) for lookup_table in lookup_tables)

### Filtering

In [23]:
def top_n(ser, n=10, sort_type='max', map_index=True):
    top_ser = {
        'min': lambda: ser.nsmallest(n=n),
        'max': lambda: ser.nlargest(n=n)
    }.get(sort_type)()

    if (map_index):
        return top_ser.index
    else:
        return top_ser.values

def get_filter_func(thresh, filter_type='gte'):
    return {
        'gte': (lambda val: val >= thresh if (val is not None and isinstance(val, float)) else False),
        'lte': (lambda val: val <= thresh if (val is not None and isinstance(val, float)) else False)
    }.get(filter_type)

### Report Viewing

In [24]:
def global_top(df, *value_dfs, n=10, sort_type='max'):
    """
    View a filtered representation of df and corresponding value dfs based on sorting method.
    
    Args:
        df (pd.DataFrame): dataframe to apply the sorting to
        *value_dfs (pd.Dataframe): dataframes to index the result of the original df into
        n (int): the top n rows returned per dataframe
        sort_type ('max'|'min'): sorting method
        
    Returns:
        dataframe of column names and values of top n according to sorting method, along with value_dfs
        indexed by the top n indices per column
    """
    top10_idx = df.iloc[:, 2:].apply(partial(top_n, n=n, sort_type=sort_type), axis=0)
    global_max_dfs = index_lookup_dfs(top10_idx, string_df_join_to_ser(df.iloc[:, :2]), df, *value_dfs)
    return global_max_dfs

## Initializing

### Corr Report Config Defaults

In [7]:
dataset_name = default_gta_dataset
test_name = default_gta_test
ret_types = ['pba_oc_return', 'pba_oa_return']
assets = ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']

### Test Defaults

In [8]:
count_tests = ['int_count', 'ratio_count']
corr_tests = ['pearson', 'spearman', 'kendall']
tests = {
    'count': count_tests,
    'corr': corr_tests
}

### *Change Config

In [9]:
dataset_name = 'eod'

### Load Metadata

In [10]:
reports_dir = sep.join([REPORT_DIR +splitext(dataset_name)[0]]) +sep
dataset_dict = load_json(dataset_name, dir_path=DATASET_DIR)

## Viewing

### Load Test Results

In [11]:
test_files = recursive_dict()

for asset in assets:
    asset_dir = sep.join([reports_dir, asset]) +sep
    files = os.listdir(asset_dir)

    for test, variants in tests.items():
        for ret_type, variant in product(ret_types, variants):
            file = getcon(getcon(files, ret_type), variant)
            test_files[asset][ret_type][test][variant] = load_df(file, dir_path=asset_dir)

### View Test Results

### Presets

In [12]:
asset_name = 'sp_500'
ret_type = 'pba_oc_return'
ratio_count_thresh = .9

In [13]:
ic = test_files[asset_name][ret_type]['count']['int_count']
rc = test_files[asset_name][ret_type]['count']['ratio_count']

prn = test_files[asset_name][ret_type]['corr']['pearson']
spr = test_files[asset_name][ret_type]['corr']['spearman']
ken = test_files[asset_name][ret_type]['corr']['kendall']

### View Test Results

In [27]:
abs_corr_df = abs_df(prn)
filtered_df = abs_corr_df[rc >= .8]

In [29]:
display(*global_max(filtered_df, rc, ic))

,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mag,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mom,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_vel,pba_oc_return_fth_af_abs_avg_shf_1_dir_conf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fb,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_1_dir_mag,pba_oc_return_fth_af_abs_avg_shf_1_dir_mom,pba_oc_return_fth_af_abs_avg_shf_1_dir_vel,pba_oc_return_fth_af_abs_avg_shf_2_dir_conf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fb,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_2_dir_mag,pba_oc_return_fth_af_abs_avg_shf_2_dir_mom,pba_oc_return_fth_af_abs_avg_shf_2_dir_vel,pba_oc_return_fth_af_abs_max_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbeod,...,vol_oc_return_fth_of_abs_xmax_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mag,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mom,vol_oc_return_fth_of_abs_xmax_shf_2_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_1_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_1_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_2_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_2_dir_vel
0,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_pba_drl_ddiff(2)_pba_open,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_pba_drl_ddiff(2)_pba_open,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_pba_drl_ddiff(2)_pba_open,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,"thresh_vol_drl_dffracdiff(0.4000,0.0100)_vol_o...",raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,"thresh_vol_drl_dffracdiff(0.4000,0.0100)_vol_o...",raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,"raw_trmi_v2_drl_dffracdiff(0.6000,0.0100)_etf_...",raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,...,"raw_trmi_v3_drl_dffracdiff(0.5000,0.0100)_etf_...",raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,"raw_trmi_v3_drl_dffracdiff(1.5000,0.0100)_mkt_...",raw_trmi_v3_mkt_v3_stockIndexOptimism_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_trmi_v3_mkt_v3_stockIndexOptimism_N,raw_trmi_v3_mkt_v3_stockIndexPriceForecast_N,raw_vol_drl_ddiff(2)_vol_open,raw_vol_drl_ddiff(2)_vol_open,raw_trmi_v3_mkt_v3_stockIndexOptimism_N,raw_vol_drl_ddiff(2)_vol_open,raw_vol_drl_ddiff(2)_vol_open,raw_vol_drl_ddiff(2)_vol_open,raw_trmi_v3_drl_ddiff(2)_etf_v3_longShort_N,raw_trmi_v2_etf_v2_managementTrust,"raw_tr

,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mag,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mom,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_vel,pba_oc_return_fth_af_abs_avg_shf_1_dir_conf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fb,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_1_dir_mag,pba_oc_return_fth_af_abs_avg_shf_1_dir_mom,pba_oc_return_fth_af_abs_avg_shf_1_dir_vel,pba_oc_return_fth_af_abs_avg_shf_2_dir_conf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fb,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_2_dir_mag,pba_oc_return_fth_af_abs_avg_shf_2_dir_mom,pba_oc_return_fth_af_abs_avg_shf_2_dir_vel,pba_oc_return_fth_af_abs_max_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbeod,...,vol_oc_return_fth_of_abs_xmax_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mag,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mom,vol_oc_return_fth_of_abs_xmax_shf_2_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_1_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_1_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_2_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_2_dir_vel
0,0.078287,0.090761,0.091609,0.089870,0.067688,0.077930,0.089654,0.065443,0.075921,0.075225,0.078506,0.054672,0.060755,0.077974,0.052685,0.053440,0.053696,0.052456,0.049253,0.049488,0.062773,0.056912,0.066380,0.064802,0.063410,...,0.047033,0.046984,0.047370,0.048099,0.051796,0.051672,0.052520,0.052610,0.056440,0.054024,0.054601,0.050134,0.049510,0.050077,0.049065,0.047898,0.050902,0.047456,0.050047,0.048979,0.049373,0.045955,0.049885,0.049641,0.050341
1,0.077787,0.088651,0.088847,0.088808,0.065501,0.074231,0.088696,0.064356,0.072918,0.073049,0.075298,0.053050,0.057525,0.073108,0.052288,0.052019,0.053401,0.050247,0.048721,0.046877,0.056182,0.054363,0.062363,0.061350,0.061255,...,0.046731,0.046338,0.046729,0.047494,0.051699,0.051599,0.052404,0.051087,0.052654,0.049325,0.051811,0.045899,0.048450,0.045663,0.048710,0.047553,0.050626,0.045806,0.049693,0.048423,0.049321,0.045805,0.049073,0.048865,0.048814
2,0.075919,0.088380,0.087309,0.088104,0.064796,0.073390,0.087095,0.062825,0.071587,0.071342,0.073906,0.052352,0.057243,0.072486,0.051183,0.051842,0.052348,0.049007,0.047583,0.046796,0.055907,0.053896,0.060461,0.059304,0.059222,...,0.046597,0.043345,0.043624,0.044938,0.051337,0.051468,0.052326,0.050680,0.051033,0.048164,0.051627,0.045614,0.046771,0.045234,0.048617,0.045190,0.047777,0.045269,0.049388,0.048232,0.049099,0.045640,0.047755,0.047464,0.048646
3,0.072308,0.085205,0.082101,0.084755,0.063559,0.073053,0.085372,0.062480,0.069290,0.067211,0.072046,0.051529,0.056370,0.071744,0.051124,0.051526,0.052324,0.045205,0.047340,0.045941,0.055805,0.052784,0.057220,0.058394,0.056903,...,0.046405,0.042992,0.043412,0.043923,0.050615,0.049876,0.050932,0.050158,0.050152,0.048014,0.049982,0.044547,0.046544,0.044392,0.047555,0.043720,0.04

,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mag,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mom,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_vel,pba_oc_return_fth_af_abs_avg_shf_1_dir_conf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fb,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_1_dir_mag,pba_oc_return_fth_af_abs_avg_shf_1_dir_mom,pba_oc_return_fth_af_abs_avg_shf_1_dir_vel,pba_oc_return_fth_af_abs_avg_shf_2_dir_conf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fb,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_2_dir_mag,pba_oc_return_fth_af_abs_avg_shf_2_dir_mom,pba_oc_return_fth_af_abs_avg_shf_2_dir_vel,pba_oc_return_fth_af_abs_max_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbeod,...,vol_oc_return_fth_of_abs_xmax_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mag,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mom,vol_oc_return_fth_of_abs_xmax_shf_2_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_1_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_1_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_2_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_2_dir_vel
0,0.997406,0.999800,0.997406,0.999800,0.997406,0.997406,0.999800,0.997406,0.997406,0.997406,0.997406,0.997406,0.997406,0.997406,0.998004,0.997406,0.998004,0.997406,0.998603,0.997406,0.997406,0.997406,0.997406,0.997406,0.997406,...,0.998204,0.997406,0.997406,0.996208,1.000000,0.997402,1.000000,0.997402,0.999600,0.999600,1.000000,0.999600,0.999600,0.999600,0.999600,1.000000,0.999800,1.000000,0.999800,0.999800,0.999800,0.999600,0.999600,0.999600,0.999800
1,0.995610,0.997406,0.995610,0.999800,0.995610,0.999800,0.997406,0.995610,0.995610,0.995610,0.995610,0.995610,0.995610,0.995610,0.998204,0.998004,0.998204,0.995610,0.998404,0.998603,0.999601,0.995610,0.995610,0.995610,0.995610,...,0.998004,0.997206,0.997206,0.996807,0.999600,1.000000,0.999600,0.995403,0.999600,0.999600,0.998001,0.999600,0.999600,0.999600,0.999600,0.999800,1.000000,0.999600,1.000000,0.999800,0.999800,0.999600,0.999800,0.999800,0.999600
2,0.997406,0.999800,0.997406,0.997406,1.000000,0.999800,0.999800,1.000000,0.997406,0.997406,0.997406,0.999800,0.999800,0.997406,0.998404,0.998204,0.999601,0.997406,0.999202,0.998404,0.997406,1.000000,0.997406,0.997406,0.997406,...,0.999002,0.997007,0.997007,1.000000,0.997402,0.999600,0.997402,0.999800,0.999600,0.999600,0.999600,0.999001,0.999600,0.999600,0.999600,0.999600,0.999600,0.999800,0.999800,1.000000,1.000000,0.999800,0.999001,0.999001,0.998001
3,1.000000,1.000000,0.999800,1.000000,0.997406,0.995610,1.000000,0.997406,1.000000,1.000000,1.000000,0.999800,0.999800,1.000000,0.999601,0.999601,0.998404,1.000000,0.998204,0.998204,0.998004,0.998204,1.000000,1.000000,1.000000,...,0.998404,0.995410,0.995410,0.994811,0.998001,0.994603,0.998001,1.000000,0.999600,0.999600,0.997402,0.998401,0.999600,0.999001,0.999800,0.998001,0.99

,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mag,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_mom,pba_oc_return_fth_af_abs_avg_shf_0.5_dir_vel,pba_oc_return_fth_af_abs_avg_shf_1_dir_conf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fb,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_1_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_1_dir_mag,pba_oc_return_fth_af_abs_avg_shf_1_dir_mom,pba_oc_return_fth_af_abs_avg_shf_1_dir_vel,pba_oc_return_fth_af_abs_avg_shf_2_dir_conf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fb,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbconf,pba_oc_return_fth_af_abs_avg_shf_2_dir_fbeod,pba_oc_return_fth_af_abs_avg_shf_2_dir_mag,pba_oc_return_fth_af_abs_avg_shf_2_dir_mom,pba_oc_return_fth_af_abs_avg_shf_2_dir_vel,pba_oc_return_fth_af_abs_max_shf_0.5_dir_conf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fb,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbconf,pba_oc_return_fth_af_abs_max_shf_0.5_dir_fbeod,...,vol_oc_return_fth_of_abs_xmax_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mag,vol_oc_return_fth_of_abs_xmax_shf_2_dir_mom,vol_oc_return_fth_of_abs_xmax_shf_2_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_0.5_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_1_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_1_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_1_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_1_dir_vel,vol_oc_return_fth_of_abs_xstd_shf_2_dir_conf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fb,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbconf,vol_oc_return_fth_of_abs_xstd_shf_2_dir_fbeod,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mag,vol_oc_return_fth_of_abs_xstd_shf_2_dir_mom,vol_oc_return_fth_of_abs_xstd_shf_2_dir_vel
0,5000.0,5011.0,5000.0,5011.0,5000.0,5000.0,5011.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,5002.0,5000.0,5002.0,5000.0,5005.0,5000.0,5000.0,5000.0,5000.0,5000.0,5000.0,...,5004.0,5000.0,5000.0,4994.0,5013.0,5000.0,5013.0,5000.0,5010.0,5010.0,5013.0,5010.0,5010.0,5010.0,5011.0,5013.0,5012.0,5013.0,5012.0,5012.0,5012.0,5011.0,5011.0,5011.0,5012.0
1,4990.0,5000.0,4990.0,5011.0,4990.0,5011.0,5000.0,4990.0,4990.0,4990.0,4990.0,4990.0,4990.0,4990.0,5003.0,5002.0,5003.0,4990.0,5004.0,5005.0,5010.0,4990.0,4990.0,4990.0,4990.0,...,5003.0,4999.0,4999.0,4997.0,5010.0,5013.0,5010.0,4990.0,5010.0,5010.0,5003.0,5010.0,5010.0,5010.0,5011.0,5012.0,5013.0,5010.0,5013.0,5012.0,5012.0,5011.0,5012.0,5012.0,5011.0
2,4999.0,5011.0,4999.0,5000.0,5013.0,5011.0,5011.0,5013.0,4999.0,4999.0,4999.0,5011.0,5011.0,4999.0,5004.0,5003.0,5010.0,4999.0,5008.0,5004.0,4999.0,5013.0,4999.0,4999.0,4999.0,...,5008.0,4998.0,4998.0,5013.0,5000.0,5010.0,5000.0,5012.0,5011.0,5011.0,5010.0,5008.0,5010.0,5010.0,5011.0,5011.0,5011.0,5012.0,5012.0,5013.0,5013.0,5012.0,5008.0,5008.0,5003.0
3,5013.0,5013.0,5011.0,5013.0,4999.0,4990.0,5013.0,4999.0,5013.0,5013.0,5013.0,5011.0,5011.0,5013.0,5010.0,5010.0,5004.0,5013.0,5003.0,5003.0,5002.0,5003.0,5013.0,5013.0,5013.0,...,5005.0,4990.0,4990.0,4987.0,5003.0,4986.0,5003.0,5013.0,5011.0,5011.0,5000.0,5005.0,5010.0,5008.0,5012.0,5003.0,5003.0,5010.0,5013.0,5013.0,5013.0,5012.0,5008.0,5008.0,5013.0
4,4998.0,5013.0,5011.0,5013.0,5003.0,4999.0,5013.0,5003.0,5003.0,5013.0,5013.0,5013.0,4999.0,5013.0,5010.0,5010.0,5010.0,5011.0,5012.0,5002.0,4990.0,4999.0,5013.0,5003.0,5003.0,...,5011.0,4998.0,4998.0,4998.0,4990.0,5003.0,4990.0,4986.0,5010.0,5004.0,5012.0,5012.0,5010.0,5011.0,5004.0,5011.0,5011.0,5011.0,5013.0,5013.0,50